In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import rma_tv, rsi_tv
from quantfreedom.plotting.plotting_base import plot_rma

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol="BTCUSDT",
    timeframe="1h",
    candles_to_dl=400,
)
close = candles[:, 4]

In [11]:
def tv_rma(
    source: np.array,
    length: int,
):
    alpha = 1 / length

    new_length = source.size - source[~np.isnan(source)].size + length

    rma = np.full_like(source, np.nan)
    rma[new_length - 1] = source[new_length - length : new_length].mean()

    for i in range(new_length, source.size):
        rma[i] = alpha * source[i] + (1 - alpha) * rma[i - 1]
    return rma

In [12]:
def rma_plot(
    candles: np.array,
    indicator: np.array,
    ind_name: str = "RMA",
    ind_color: str = "yellow",
):
    datetimes = pd.to_datetime(candles[:, 0], unit="ms")
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=datetimes,
                open=candles[:, 1],
                high=candles[:, 2],
                low=candles[:, 3],
                close=candles[:, 4],
                name="Candles",
            ),
            go.Scatter(
                x=datetimes,
                y=indicator,
                name=ind_name,
                line_color=ind_color,
            ),
        ]
    )
    fig.update_layout(height=800, xaxis_rangeslider_visible=False)
    fig.show()

In [17]:
tv_rma(source=close, length=20)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, 35028.04, 35023.38, 35019.72,
       35010.46, 35005.82, 35000.75, 34996.44, 34994.62, 34992.11,
       34988.57, 34978.74, 34961.55, 34944.85, 34934.07, 34926.01,
       34920.59, 34910.59, 34898.32, 34896.05, 34919.25, 34945.64,
       34977.11, 35002.20, 35016.83, 35035.27, 35051.28, 35067.59,
       35080.08, 35089.73, 35101.60, 35109.61, 35117.33, 35123.51,
       35132.09, 35140.82, 35153.72, 35162.66, 35173.95, 35184.87,
       35191.44, 35193.74, 35203.93, 35209.26, 35228.11, 35247.75,
       35266.18, 35282.13, 35307.35, 35323.42, 35350.44, 35378.67,
       35425.38, 35474.11, 35531.27, 35581.71, 35640.67, 35686.33,
       35735.52, 35791.24, 35839.13, 35887.62, 35937.49, 35999.60,
       36094.27, 36145.32, 36155.55, 36179.88, 36190.21, 36199.37,
       36218.40, 36233.52, 36249.82, 36271.36, 36284.69, 36303.97,
       36332.13, 36352.42, 36374.68, 36393.89, 36405.02, 

In [18]:
rma_plot(candles=candles, indicator=tv_rma(source=close, length=20))